In [1]:
import os

import astropy.coordinates as coord
from astropy.coordinates.matrix_utilities import rotation_matrix
import astropy.table as at
from astropy.io import fits
import astropy.units as u
import gala.coordinates as gc

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline
import numpy as np
import healpy as hp
from scipy.optimize import minimize
from tqdm import tqdm, trange
from galstreams import MWStreams

from helpers import projs, get_data, apwnorm

def func(x, y, z, n=None):
    if n is None:
        n = nside
    return hp.vec2pix(n, x, y, z)

In [2]:
def obj_func(ang, icrs, R):
    R3 = rotation_matrix(ang, 'x')
    rep = icrs.cartesian.transform(R3 @ R).represent_as(
        coord.UnitSphericalRepresentation)
    return np.sum(rep.lat.degree**2)

def get_roll(icrs, R, x0=0.):
    res = minimize(obj_func, x0=x0, args=(icrs, R))
    return coord.Angle(res.x[0] * u.deg)

def get_origin(fr):
    usph_origin = coord.UnitSphericalRepresentation(0*u.deg, 0*u.deg)
    return fr.realize_frame(usph_origin).transform_to(coord.ICRS)

def get_rot(fr, x0=0.):
    origin = get_origin(fr)
    usph_band  = coord.UnitSphericalRepresentation(np.random.uniform(0, 360, 10000)*u.deg, 
                                                   np.random.uniform(-1, 1, size=10000)*u.deg)
    band_icrs = fr.realize_frame(usph_band).transform_to(coord.ICRS)
    
    R1 = rotation_matrix(origin.ra, 'z')
    R2 = rotation_matrix(-origin.dec, 'y')
    Rtmp = R2 @ R1
    
    roll = get_roll(band_icrs, Rtmp, x0)
    
    return [origin.ra.degree, origin.dec.degree, roll.degree]

### Load data

In [4]:
cube, dm, footprint_mask = get_data(bass_file='../data/bass_dr8_iso_hpxcube_v0.fits.gz', 
                                    decals_file='../data/decals_dr8_iso_hpxcube_v4.fits.gz', 
                                    bass_scale_factor=1.3)
npix, nslice = cube.shape
nside = hp.npix2nside(npix)

In [5]:
pix_idx = np.arange(hp.nside2npix(nside), dtype=int)
pix_c = hp.pix2ang(nside, pix_idx, 
                   lonlat=True)
pix_c = coord.SkyCoord(pix_c[0]*u.deg, pix_c[1]*u.deg)

In [6]:
pix_gal = pix_c.transform_to(coord.Galactic)
pix_l = pix_gal.l
pix_b = pix_gal.b

### Make maps in stream coordinates

In [7]:
streams = MWStreams(verbose=False)
streams.keys()

/Users/apricewhelan/projects/astropy/astropy/units/quantity.py:481: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


dict_keys(['VOD/VSS', 'Monoceros', 'EBS', 'Her-Aq', 'PAndAS', 'Tri-And', 'Tri-And2', 'PiscesOv', 'EriPhe', 'Phoenix', 'WG1', 'WG2', 'WG3', 'WG4', 'Acheron', 'Cocytos', 'Lethe', 'Styx', 'ACS', 'Pal15', 'Eridanus', 'TucanaIII', 'Indus', 'Jhelum', 'Ravi', 'Chenab', 'Elqui', 'Aliqa_Uma', 'Turbio', 'Willka_Yaku', 'Turranburra', 'Wambelong', 'Palca', 'Jet', 'Gaia-1', 'Gaia-2', 'Gaia-3', 'Gaia-4', 'Gaia-5', 'PS1-A', 'PS1-B', 'PS1-C', 'PS1-D', 'PS1-E', 'ATLAS', 'Ophiucus', 'Sangarius', 'Scamander', 'Corvus', '20.0-1', 'Sgr-L10', 'Orphan', 'Pal5', 'GD-1', 'Tri/Pis', 'NGC5466', 'Alpheus', 'Hermus', 'Hyllus', 'Cetus', 'Kwando', 'Molonglo', 'Murrumbidgee', 'Orinoco', 'Phlegethon', 'Slidr', 'Sylgr', 'Ylgr', 'Fimbulthul', 'Svol', 'Fjorm', 'Gjoll', 'Leiptr'])

In [33]:
streams_to_plot = {
    'ACS': (95., 0.15, 2),
    'ATLAS': (95., 0.1, 1),
    'Pal5': (99.5, 0.1, 1),
    'GD-1': (95., 0.12, 2),
    'Orphan': (95., 0.15, 2),
    'Jhelum': (95., 0.15, 1),
    'Phoenix': (95., 0.1, 1),
    'Tri/Pis': (95., 0.1, 1),
    'TucanaIII': (95., 0.1, 1)
}

In [34]:
# stream = streams['ATLAS']
# frame = stream.gcfr
# cc = coord.SkyCoord(stream.ra*u.deg, stream.dec*u.deg).transform_to(frame)
# # plt.scatter(cc.phi1,cc.phi2)
# np.min(cc.phi1.degree), np.max(cc.phi1.degree), np.min(cc.phi2.degree), np.max(cc.phi2.degree)

In [36]:
# for key in streams.keys():
for key, (perc, smooth, fac) in streams_to_plot.items():
    # if key != 'Jhelum': 
    #     continue
        
    stream = streams[key]
    
    name = key.replace('/', '').replace('-', '')
    if hasattr(gc, name):
        stream_fr = getattr(gc, name.replace('-', ''))()
    else:
        stream_fr = stream.gcfr

    if any(stream.Rhel < 0):
        continue
        
    if stream.Rhel.max() > stream.Rhel.min():
        stream_dm = coord.Distance([stream.Rhel.min(), 
                                    stream.Rhel.max()]*u.kpc).distmod.value
    else:
        stream_dm = coord.Distance(stream.Rhel.min()*u.kpc).distmod.value
        stream_dm = np.array([stream_dm - 1, stream_dm + 1])

    # Evenly spaced in distance modulus:
    dist_idx = np.linspace(np.argmin(np.abs(stream_dm.min() - dm)), 
                           np.argmin(np.abs(stream_dm.max() - dm)),
                           4).astype(int)

    rgb_idx = [(dist_idx[2], dist_idx[3]), 
               (dist_idx[1], dist_idx[2]),
               (dist_idx[0], dist_idx[1])]

    origin = get_origin(stream_fr)

    stream_ctr_map = np.zeros(npix, dtype=bool)
    stream_ctr_map[hp.ang2pix(nside, 
                              origin.ra.degree, 
                              origin.dec.degree, 
                              lonlat=True)] = True

    rgb = [cube[:, a:b].sum(axis=1) 
           for a, b in rgb_idx]
    rgb = [hp.smoothing(x, sigma=np.radians(smooth))
           for x in rgb]

    rgb = [apwnorm(rgb[i], 
                   min=np.nanpercentile(rgb[i], 1),
                   max=np.nanpercentile(rgb[i], perc))
           for i in range(3)]

    make_plots = (stream_ctr_map & footprint_mask).any()
    # if not make_plots or os.path.exists(f'../plots/individual/{name}-moll.png'):
    #     continue

    proj = hp.projector.MollweideProj(rot=get_rot(stream_fr), 
                                      xsize=fac*1024)
    img = np.stack([proj.projmap(rgb[i], func) 
                    for i in range(3)], axis=-1)
    
    # ---
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    ax.imshow(img, origin='bottom', extent=proj.get_extent(),
              vmax=np.nanpercentile(img, perc), cmap='Greys')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    fig.savefig(f'../plots/individual/{name}-moll.png', dpi=300)
    plt.close(fig)

    stream_fr_c = coord.SkyCoord([stream.end_o.ra.degree, stream.end_f.ra.degree]*u.deg, 
                                 [stream.end_o.dec.degree, stream.end_f.dec.degree]*u.deg).transform_to(stream_fr)

    off_plus_c = coord.SkyCoord(stream_fr_c.phi1, 
                                stream_fr_c.phi2 + 3*u.deg,
                                frame=stream_fr)
    off_minu_c = coord.SkyCoord(stream_fr_c.phi1, 
                                stream_fr_c.phi2 - 3*u.deg,
                                frame=stream_fr)

    proj = hp.projector.GnomonicProj(rot=get_rot(stream_fr), 
                                     xsize=fac*1024, ysize=512,
                                     reso=3.)
    img = np.stack([proj.projmap(rgb[i], func) 
                    for i in range(3)], axis=-1)
    
    # ---
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    ax.imshow(img, origin='bottom', extent=proj.get_extent(),
              vmax=np.nanpercentile(img, perc), cmap='Greys')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    fig.savefig(f'../plots/individual/{name}-streamcoord.png', dpi=300)

    for cc in [off_plus_c, off_minu_c]:
        x, y = proj.ang2xy(cc.icrs.ra.degree,
                           cc.icrs.dec.degree, 
                           lonlat=True)
        ax.plot(x, y, color='tab:red', marker='', lw=1.)

    xlim = proj.get_extent()[:2]
    ylim = proj.get_extent()[2:]
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    fig.savefig(f'../plots/individual/{name}-streamcoord-line.png', dpi=300)
    plt.close(fig)
    
    continue
    # ---
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    fig.tight_layout()

    im = ax.imshow(proj.projmap(np.zeros(hp.nside2npix(nside)), func), 
                   cmap='Greys', origin='bottom', extent=proj.get_extent(),
                   zorder=50)

    def plot_slice(i):
        X = hp.smoothing(cube[:, i], sigma=np.radians(0.08))
        X = apwnorm(X) 
        X[~footprint_mask] = 0.

        im.set_array(proj.projmap(X, func))

        vmin = np.percentile(X, 1),
        vmax = np.percentile(X, perc)
        im.norm.vmin = vmin
        im.norm.vmax = vmax
        return im,

    frames = np.arange(max(0, np.min(rgb_idx) - 5), 
                       min(nslice-1, np.max(rgb_idx) + 5))
    anim = FuncAnimation(fig, plot_slice, frames=frames,
                         blit=True)
    anim.save(f'../plots/individual/{name}-movie.mp4', dpi=250)
    plt.close(fig)

Sigma is 9.000000 arcmin (0.002618 rad) 
-> fwhm is 21.193380 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 9.000000 arcmin (0.002618 rad) 
-> fwhm is 21.193380 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 9.000000 arcmin (0.002618 rad) 
-> fwhm is 21.193380 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 6.000000 arcmin (0.001745 rad) 
-> fwhm is 14.128920 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 6.000000 arcmin (0.001745 rad) 
-> fwhm is 14.128920 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 6.000000 arcmin (0.001745 rad) 
-> fwhm is 14.128920 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 6.000000 arcmin (0.001745 rad) 
-> fwhm is 14.128920 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 6.000000 arcmin (0.001745 rad) 


/Users/apricewhelan/projects/refind-the-streams/notebooks/helpers.py:13: RuntimeWarning: invalid value encountered in true_divide
  return np.clip((x - min) / (max - min), 0, 1)


In [166]:
# COORDINATE GRID

# nband = 128
# color = '#ffffff'
# ls = '--'
# for lon in np.arange(0, 360+1e-3, 10):
#     x, y = proj.ang2xy(np.full(nband, lon), np.linspace(-90, 90, nband), lonlat=True)
#     mask = (x > xlim[1]) | (x < xlim[0]) | (y > ylim[1]) | (y < ylim[0])
#     x[mask] = np.nan
#     y[mask] = np.nan
#     ax.plot(x, y, marker='', zorder=100, color=color, ls=ls, alpha=0.75)
    
#     if np.any(np.isfinite(x)) and np.any(np.isfinite(y)):
#         i = np.nanargmin(y)
#         ax.text(x[i], y[i], r'${:.0f}^\circ$'.format(lon), 
#                 fontsize=15, color='w', va='bottom', ha='left')

# for lat in np.arange(-90, 90+1e-3, 10):
#     x, y = proj.ang2xy(np.linspace(0, 360, nband), np.full(nband, lat), lonlat=True)
#     mask = (x > xlim[1]) | (x < xlim[0]) | (y > ylim[1]) | (y < ylim[0])
#     x[mask] = np.nan
#     y[mask] = np.nan
#     ax.plot(x, y, marker='', zorder=100, color=color, ls=ls, alpha=0.75)
    
#     if np.any(np.isfinite(x)) and np.any(np.isfinite(y)):
#         i = np.nanargmax(x)
#         ax.text(x[i], y[i], r'${:.0f}^\circ$'.format(lat), 
#                 fontsize=15, color='w', va='bottom', ha='right')